In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta

df = pd.read_csv('bigdata.csv')
# df = pd.read_csv('bigdata_time.csv')

In [3]:
df.head()

,Time,PRICE,SIZE,EXCHANGE
0,10:00:00.009,135.14,100,V
1,10:00:00.010,135.14,100,V
2,10:00:00.013,135.13,100,D
3,10:00:00.015,135.13,300,D
4,10:00:00.016,135.13,150,D


In [4]:
# df['Time'] = datetime.strptime(df['Time'].iloc[1], '%H:%M:%S.%f')
j = 0
for i in df['Time']:
    df['Time'][j] = datetime.strptime(i, '%H:%M:%S.%f')
    j += 1

# df['Time'] =df['Time'].strftime('%H:%M:%S.%f')
# n = df['Time'].strftime('%H:%M:%S.%f')
# n
# df['Time']

C:\Users\ANDREY.FENOMEN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
df.to_csv('bigdata_time.csv')

In [6]:
df.shape

(9993, 4)

In [7]:
timer = df['Time'][9992] - df['Time'][0]
# t1.year, t1.month, t1.day, t1.hour, t1.minute, t1.second
timer

datetime.timedelta(0, 958, 597000)

##### Количество сделок за промежуток времени 12:01:08.104 и 12:01:09.103

Функция по расчёту максимального количества сделок

In [118]:
def maxtrans(start, stop, df=df):
    return df[(df['Time'] >= start) & (df['Time'] <= stop)]['EXCHANGE'].count()

In [9]:
s = maxtrans(datetime(1900, 1, 1, 10, 1, 8, 104), datetime(1900, 1, 1, 10, 1, 9, 103))
s

23

##### Групировка количества сделок по биржам

Функция группировки сделок по биржам

In [100]:
def drouped(start, stop, df=df):
    return df[(df['Time'] >= start) & (df['Time'] <= stop)].groupby(df['EXCHANGE'])['EXCHANGE'].count()

# drouped = df[(df['Time'] >= datetime(1900, 1, 1, 10, 1, 8, 104)) & (df['Time'] <= datetime(1900, 1, 1, 10, 1, 9, 103))].groupby(df['EXCHANGE'])['EXCHANGE'].count()

##### Предварительный "грубый" поиск максимального количества сделог (шаг 1 секунда)

In [124]:
delta = timedelta(seconds = 1)

def firstparse(delta, df=df, t1=0, t2=9992):
    timer = df['Time'][t2] - df['Time'][t1]
#     print('timer', timer.seconds)
    timestart = df['Time'][t1]
    timeend = df['Time'][t2]
    transcount = 0

    bigtimestart = 0

    for i in range(timer.seconds):
        timestep = timestart + delta
        if timestep <= timeend:
            n = maxtrans(timestart, timestep, df)
            if n > transcount:
                transcount = n
                bigtimestart = timestart
#                 print(transcount, bigtimestart)
        else:
            break
        timestart = timestep
    print(transcount, bigtimestart)
    return bigtimestart

In [125]:
firstparse(delta)

479 1900-01-01 10:02:00.009000


datetime.datetime(1900, 1, 1, 10, 2, 0, 9000)

In [68]:
def shaping(start, stop):
    n = df[(df['Time'] >= start) & (df['Time'] <= stop)].shape[0]
    return n

##### Уточняющий расчет в районе промежутся с нужным выбросом большого количества сделок. (шаг 100 микросекунд)

In [104]:
def prog_micro(bigtimestart, delta, microdel, df=df):
    delta = timedelta(seconds = delta)
    transcount = 0
    bigtime = 0
    delta1 = timedelta(microseconds = microdel)
#     print(delta1)
    start1 = bigtimestart - delta
#     print(start1)
    stop1 = bigtimestart + delta * 2
#     print(stop1)
    timer1 = stop1 - start1
#     print(timer1.seconds)
    
#     print(int(timer1/delta1))
    
    for i in range(int(timer1/delta1)):
        time_interval = start1 + delta1 + delta
#         print('time_interval',time_interval)
        if time_interval <= stop1:
            n = drouped(start1, time_interval, df).sum()
            if n > transcount:
                transcount = n
                bigtime = start1 + delta1
                bigend = time_interval
#                 print(transcount, bigtime, bigend, bigend-bigtime)
        else:
            break
        start1 = start1 + delta1
    print(transcount, bigtime, bigend, bigend-bigtime)
    return bigtime, bigend

In [106]:
prog_micro(bigtimestart, 1, 100)
# %time

516 1900-01-01 10:02:00.734000 1900-01-01 10:02:01.734000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 2, 0, 734000),
 datetime.datetime(1900, 1, 1, 10, 2, 1, 734000))

##### Максимальное найденое число сделок за 1 секунду равняется 516 в промежутке 10:02:00.734000 10:02:01.734000

Максимальное количество сделок (в количестве 516) в течение одной секунды было между 10:02:00.734000 и 10:02:01.734000

##### Выводим полученное количество сделок, сгруппированное по биржам:

In [107]:
drouped(datetime(1900, 1, 1, 10, 2, 0, 734000), datetime(1900, 1, 1, 10, 2, 1, 734000))

EXCHANGE
B      1
D     47
J      2
K    133
P     41
Q    239
X      9
Y      1
Z     43
Name: EXCHANGE, dtype: int64

##### Выделяем общее количество сделок по каждой бирже

In [110]:
B = df[df['EXCHANGE'] == 'B']
D = df[df['EXCHANGE'] == 'D']
J = df[df['EXCHANGE'] == 'J']
K = df[df['EXCHANGE'] == 'K']
P = df[df['EXCHANGE'] == 'P']
Q = df[df['EXCHANGE'] == 'Q']
X = df[df['EXCHANGE'] == 'X']
Y = df[df['EXCHANGE'] == 'Y']
Z = df[df['EXCHANGE'] == 'Z']

B.shape[0], D.shape[0], J.shape[0], K.shape[0], P.shape[0], Q.shape[0], X.shape[0], Y.shape[0], Z.shape[0]

(279, 2431, 184, 1472, 1363, 2636, 193, 302, 973)

In [93]:
#B # 14 - 9852
# D # 2 - 9992
# J # 17 - 9972
# K # 34 - 9988
# P # 45 - 9989
# Q # 27 - 9980
# X # 11 - 9977
# Y # 12 - 9875
# Z # 32 - 9984

##### Биржа 'B'. Рассчет максимального числа сделок за 1 секунду.

In [120]:
firstparse(delta, B, 14, 9852)

timer 947
10 1900-01-01 10:13:00.877000


datetime.datetime(1900, 1, 1, 10, 13, 0, 877000)

In [122]:
prog_micro(datetime(1900, 1, 1, 10, 13, 0, 877000), 1, 100, B)

11 1900-01-01 10:13:00.301000 1900-01-01 10:13:01.301000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 13, 0, 301000),
 datetime.datetime(1900, 1, 1, 10, 13, 1, 301000))

Максимальное количество сделок на бирже 'B' (в количестве 11) в течение одной секунды было между 10:13:00.301000 и 10:13:01.301000

##### Биржа 'D'. Рассчет максимального числа сделок за 1 секунду.

In [127]:
firstparse(delta, D, 2, 9992)

47 1900-01-01 10:02:00.013000


datetime.datetime(1900, 1, 1, 10, 2, 0, 13000)

In [128]:
prog_micro(datetime(1900, 1, 1, 10, 2, 0, 13000), 1, 100, D)

47 1900-01-01 10:01:59.919000 1900-01-01 10:02:00.919000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 1, 59, 919000),
 datetime.datetime(1900, 1, 1, 10, 2, 0, 919000))

Максимальное количество сделок на бирже 'D' (в количестве 47) в течение одной секунды было между 10:01:59.919000 и 10:02:00.919000

##### Биржа 'J'. Рассчет максимального числа сделок за 1 секунду.

In [130]:
firstparse(delta, J, 17, 9972)

8 1900-01-01 10:09:54.263000


datetime.datetime(1900, 1, 1, 10, 9, 54, 263000)

In [131]:
prog_micro(datetime(1900, 1, 1, 10, 9, 54, 263000), 1, 100, J)

8 1900-01-01 10:09:53.387000 1900-01-01 10:09:54.387000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 9, 53, 387000),
 datetime.datetime(1900, 1, 1, 10, 9, 54, 387000))

Максимальное количество сделок на бирже 'J' (в количестве 8) в течение одной секунды было между 10:09:53.387000 и 10:09:54.387000

##### Биржа 'K'. Рассчет максимального числа сделок за 1 секунду.

In [132]:
# K # 34 - 9988
firstparse(delta, K, 34, 9988)

133 1900-01-01 10:02:00.717000


datetime.datetime(1900, 1, 1, 10, 2, 0, 717000)

In [133]:
prog_micro(datetime(1900, 1, 1, 10, 2, 0, 717000), 1, 100, K)

133 1900-01-01 10:02:00.449000 1900-01-01 10:02:01.449000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 2, 0, 449000),
 datetime.datetime(1900, 1, 1, 10, 2, 1, 449000))

Максимальное количество сделок на бирже 'K' (в количестве 133) в течение одной секунды было между 10:02:00.44900 и 10:02:01.449000

##### Биржа 'P'. Рассчет максимального числа сделок за 1 секунду.

Предварительный расчёт с шагом 1 секунда:

In [135]:
# P # 45 - 9989
firstparse(delta, P, 45, 9989)

52 1900-01-01 10:14:23.718000


datetime.datetime(1900, 1, 1, 10, 14, 23, 718000)

Уточняющий расчёт с шагом 100 микросекунд:

In [136]:
prog_micro(datetime(1900, 1, 1, 10, 14, 23, 718000), 1, 100, P)

55 1900-01-01 10:14:23.482000 1900-01-01 10:14:24.482000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 14, 23, 482000),
 datetime.datetime(1900, 1, 1, 10, 14, 24, 482000))

Максимальное количество сделок на бирже 'P' (в количестве 55) в течение одной секунды было между 10:14:23.482000 и 10:14:24.482000

##### Биржа 'Q'. Рассчет максимального числа сделок за 1 секунду.

In [140]:
# Q # 27 - 9980
firstparse(delta, Q, 27, 9980)

231 1900-01-01 10:02:00.717000


datetime.datetime(1900, 1, 1, 10, 2, 0, 717000)

In [141]:
prog_micro(datetime(1900, 1, 1, 10, 2, 0, 717000), 1, 100, Q)

243 1900-01-01 10:02:00.109000 1900-01-01 10:02:01.109000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 2, 0, 109000),
 datetime.datetime(1900, 1, 1, 10, 2, 1, 109000))

Максимальное количество сделок на бирже 'Q' (в количестве 243) в течение одной секунды было между 10:02:00.109000 и 10:02:01.109000

##### Биржа 'X'. Рассчет максимального числа сделок за 1 секунду.

In [143]:
# X # 11 - 9977
firstparse(delta, X, 11, 9977)

7 1900-01-01 10:02:01.469000


datetime.datetime(1900, 1, 1, 10, 2, 1, 469000)

In [144]:
prog_micro(datetime(1900, 1, 1, 10, 2, 1, 469000), 1, 100, X)

9 1900-01-01 10:02:00.694000 1900-01-01 10:02:01.694000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 2, 0, 694000),
 datetime.datetime(1900, 1, 1, 10, 2, 1, 694000))

Максимальное количество сделок на бирже 'X' (в количестве 9) в течение одной секунды было между 10:02:00.694000 и 10:02:01.694000

##### Биржа 'Y'. Рассчет максимального числа сделок за 1 секунду.

In [145]:
# Y # 12 - 9875
firstparse(delta, Y, 12, 9875)

12 1900-01-01 10:09:54.045000


datetime.datetime(1900, 1, 1, 10, 9, 54, 45000)

In [147]:
prog_micro(datetime(1900, 1, 1, 10, 9, 54, 45000), 1, 100, Y)

12 1900-01-01 10:09:53.391000 1900-01-01 10:09:54.391000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 9, 53, 391000),
 datetime.datetime(1900, 1, 1, 10, 9, 54, 391000))

Максимальное количество сделок на бирже 'Y' (в количестве 12) в течение одной секунды было между 10:09:53.391000 и 10:09:54.391000

##### Биржа 'Z'. Рассчет максимального числа сделок за 1 секунду.

In [151]:
# Z # 32 - 9984
firstparse(delta, Z, 32, 9984)

43 1900-01-01 10:02:00.717000


datetime.datetime(1900, 1, 1, 10, 2, 0, 717000)

In [152]:
prog_micro(datetime(1900, 1, 1, 10, 2, 0, 717000), 1, 100, Z)

43 1900-01-01 10:02:00.110000 1900-01-01 10:02:01.110000 0:00:01


(datetime.datetime(1900, 1, 1, 10, 2, 0, 110000),
 datetime.datetime(1900, 1, 1, 10, 2, 1, 110000))

Максимальное количество сделок на бирже 'Z' (в количестве 43) в течение одной секунды было между 10:02:00.110000 и 10:02:01.110000